In [ ]:
!pip install git+https://github.com/SberProcessMining/Sber_Process_Mining.git
!pip install pm4py

In [1]:
import sberpm
import pandas as pd
from sberpm.miners import SimpleMiner,CausalMiner, HeuMiner,AlphaPlusMiner, AlphaMiner, InductiveMiner
from sberpm.miners._abstract_miner import AbstractMiner
%matplotlib inline
from sberpm.visual import GraphvizPainter
import matplotlib.pyplot as plt
from sberpm import DataHolder
import networkx as nx
from sberpm.metrics import ActivityMetric, TransitionMetric, IdMetric, TraceMetric, UserMetric,TokenReplay
import pm4py

In [2]:
data = pd.read_excel('/content/drive/MyDrive/Drill planner/Data/Dataset2.xlsx')

In [3]:
data

,Unnamed: 0,Well number,Section,Phase,Trip number,Data,"Depth, m","Time, h","Time, days","Time, h (in grains)",Type of work,Operation code,Operation description,Timestamp,"ROP, m/h"
0,0.0,2083.0,Направление,БУРНАП,1.0,2021-08-06,0.0,02:00:00,0.083333,2.000000,Сборка роторной КНБК,301.0,Сборка КНБК,2022-01-01 00:00:00,NaN
1,1.0,2083.0,Направление,БУРНАП,1.0,2021-08-06,41.9,07:50:00,0.326389,7.833333,Бурение с вращением без ВЗД,204.0,"Бурение в инт. 10.3-41.9м (G=2-5т, Qвх=20-50л/...",2022-01-01 07:50:00,5.348936
2,2.0,2083.0,Направление,БУРНАП,1.0,2021-08-06,41.9,02:00:00,0.083333,2.000000,Проработка перед наращиванием,419.0,Проработка,2022-01-01 09:50:00,NaN
3,3.0,2083.0,Направление,БУРНАП,1.0,2021-08-06,41.9,01:20:00,0.055556,1.333333,Наращивание во время бурения трубкой (с мостков),304.0,Наращивание (3раза),2022-01-01 11:10:00,NaN
4,4.0,2083.0,Направление,БУРНАП,1.0,2021-08-06,41.9,00:25:00,0.017361,0.416667,Шаблонировка вверх,323.0,Подъем инструмента (шаблонирование) в инт. 31....,2022-01-01 11:35:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5733,6657.0,3094.0,Хвостовик,КРЕПХВОСТ,NaN,2022-05-03,4780.0,00:20:00,0.013889,0.333333,Промывка без расхаживания с вращением БИ,402.0,ПР к вымыву ДТ.,2022-03-26 01:20:00,NaN
5734,6658.0,3094.0,Хвостовик,КРЕПХВОСТ,NaN,2022-05-03,4780.0,01:15:00,0.052083,1.583333,Перевод на жидкость заканчивания,411.0,Обратная промывка через ЦА 320. Вымыв ДТ. (Qвх...,2022-03-26 02:35:00,NaN
5735,6661.0,3094.0,Хвостовик,КРЕПХВОСТ,NaN,2022-05-04,4780.0,01:05:00,0.045139,1.083333,Промывка без расхаживания и вращения БИ,403.0,"Промывка, перевод скважины на KCl плотностью 1...",2022-03-26 04:30:00,NaN
5736,6662.0,3094.0,Хвостовик,КРЕПХВОСТ,NaN,2022-05-04,4780.0,10:30:00,0.437500,10.500000,Подъем БИ с выбросом на мостки в обсаженном ст...,326.0,Подъем транспортировочной колонны в инт. 3548....,2022-03-26 15:00:00,NaN


In [4]:
data['Operation code'] = data['Operation code'].astype(int)
burek = data[data['Phase'] == 'БУРЭК']
burek['Operation code'] = burek['Operation code'].astype(int)
data_holder = DataHolder(data=burek, 
                         id_column='Well number', 
                         activity_column='Operation code', 
                         start_timestamp_column='Timestamp', 
                         time_format='%Y.%m.%d')

#pd.set_option("max_rows",1000)
#burek

In [5]:
simple_miner = SimpleMiner(data_holder)
simple_miner.apply()
graph = simple_miner.graph

# Visualization
painter = GraphvizPainter()
painter.apply(graph)
painter.show()
#painter.write_graph("alpha_miner.png", format='png')

In [6]:
# Miner
causal_miner = CausalMiner(data_holder)
simple_miner = SimpleMiner(data_holder)
heu_miner = HeuMiner(data_holder)
alpha_plus_miner = AlphaPlusMiner(data_holder)
inductive_miner = InductiveMiner(data_holder)
alpha_miner = AlphaMiner(data_holder)

miners = [causal_miner,simple_miner,heu_miner,alpha_plus_miner] #alpha_miner]
#pure_miners = [CausalMiner,SimpleMiner,HeuMiner,AlphaPlusMiner,AlphaMiner]#InductiveMiner]

for i in miners:
  i.apply()

graph = simple_miner.graph


# Visualization
painter = GraphvizPainter()
painter.apply(simple_miner.graph)
painter.show()

In [ ]:
from sberpm.metrics import ActivityMetric, TransitionMetric, IdMetric, TraceMetric, UserMetric

In [ ]:
activity_metric = ActivityMetric(data_holder, time_unit='day')
activity_metric.apply().head()

,count,unique_ids,unique_ids_num,aver_count_in_trace,loop_percent,throughput,total_duration,mean_duration,median_duration,max_duration,min_duration,variance_duration,std_duration
401,24,"{3094, 3024, 1051, 3145, 3044, 2057, 2076, 2084}",8,3.000000,66.666667,9.770992,2.456250,0.102344,0.062500,0.362500,0.013889,647.645182,0.086579
425,17,"{3024, 1051, 3145, 2057, 2076, 2084}",6,2.833333,64.705882,21.012876,0.809028,0.047590,0.017361,0.166667,0.003472,211.245675,0.049447
318,16,"{1051, 3145, 3044, 2057, 2076, 2084}",6,2.666667,62.500000,17.066667,0.937500,0.058594,0.034722,0.295139,0.006944,512.093099,0.076987
706,16,"{3094, 3024, 1051, 3145, 3044, 3063, 2057, 207...",9,1.777778,43.750000,1.119534,14.291667,0.952778,1.000000,2.000000,0.364583,15611.361111,0.425073
422,14,"{3024, 1051, 3145, 3044, 3094, 2076, 2084}",7,2.000000,50.000000,5.448649,2.569444,0.183532,0.114583,0.947917,0.020833,4280.782313,0.222590


In [ ]:
transition_metric = TransitionMetric(data_holder, time_unit='day')
transition_metric.apply().head()

,count,unique_ids,unique_ids_num,aver_count_in_trace,loop_percent,throughput,total_duration,mean_duration,median_duration,max_duration,min_duration,variance_duration,std_duration
"(606, 422)",12,"{3024, 3145, 3044, 3094, 2076, 2084}",6,2.000000,50.000000,4.287841,2.798611,0.233218,0.121528,0.854167,0.031250,5008.304398,0.240762
"(703, 706)",12,"{3024, 3145, 3044, 2057, 2076, 2084}",6,2.000000,50.000000,2.052257,5.847222,0.487269,0.592014,0.819444,0.034722,6978.703704,0.284204
"(701, 703)",7,"{3024, 3145, 3044, 2057, 2076, 2084}",6,1.166667,14.285714,10.610526,0.659722,0.094246,0.097222,0.156250,0.020833,122.448980,0.037646
"(306, 601)",7,"{3094, 3024, 1051, 3145, 2057, 2076, 2084}",7,1.000000,0.000000,7.148936,0.979167,0.139881,0.135417,0.263889,0.034722,453.188776,0.072424
"(601, 606)",6,"{3094, 3024, 3044, 3063, 2057, 2076}",6,1.000000,0.000000,2.307076,2.600694,0.433449,0.414931,0.791667,0.215278,2902.228009,0.183277


In [ ]:
id_metric = IdMetric(data_holder, time_unit='day')
id_metric.apply()

,trace,trace_length,unique_activities,unique_activities_num,loop_percent,total_duration,mean_duration,median_duration,max_duration,min_duration,variance_duration,std_duration
1051,"(301, 425, 301, 425, 301, 317, 425, 324, 317, ...",58,"{805, 1002, 302, 702, 201, 317, 348, 801, 704,...",26,55.172414,6.225694,0.109223,0.024306,1.250000,0.003472,3801.939699,0.209771
2057,"(301, 425, 301, 324, 401, 315, 306, 601, 606, ...",41,"{805, 407, 406, 1002, 905, 302, 410, 201, 703,...",29,29.268293,7.277778,0.181944,0.088542,1.097222,0.010417,4922.437500,0.238689
2076,"(301, 317, 401, 315, 407, 315, 306, 601, 606, ...",51,"{346, 805, 407, 406, 1002, 302, 201, 317, 1010...",30,41.176471,8.916667,0.178333,0.085069,1.020833,0.006944,5659.656667,0.255940
2084,"(301, 1012, 301, 425, 346, 301, 425, 403, 323,...",51,"{346, 805, 1002, 302, 410, 201, 703, 432, 403,...",31,39.215686,8.152778,0.163056,0.072917,1.437500,0.003472,5893.040000,0.261164
3024,"(301, 317, 425, 331, 401, 315, 425, 306, 601, ...",20,"{805, 601, 331, 801, 330, 315, 425, 301, 706, ...",16,20.000000,6.333333,0.333333,0.142361,1.038194,0.020833,11615.241228,0.366655
3044,"(601, 606, 422, 606, 422, 701, 703, 706, 703, ...",35,"{805, 343, 1002, 802, 410, 201, 432, 703, 701,...",23,34.285714,8.482639,0.249489,0.085069,1.548611,0.006944,12634.505659,0.382404
3063,"(601, 606, 406, 701, 702, 706)",6,"{601, 406, 706, 702, 701, 606}",6,0.000000,1.215278,0.243056,0.152778,0.642361,0.065972,3638.750000,0.205220
3094,"(301, 317, 401, 323, 401, 407, 315, 407, 315, ...",18,"{601, 801, 323, 407, 315, 301, 706, 702, 306, ...",15,16.666667,5.362500,0.315441,0.159722,2.000000,0.017361,17780.585928,0.453645
3145,"(301, 317, 401, 315, 306, 601, 618, 606, 422, ...",37,"{805, 103, 1002, 302, 410, 201, 317, 703, 801,...",26,29.729730,8.753472,0.243152,0.121528,1.041667,0.013889,7245.803594,0.289592


In [ ]:
token_replay = TokenReplay(data_holder, simple_miner.graph)
token_replay.apply()
token_replay.result

NameError: ignored

In [ ]:
# Calculation of the metrics
nodes_count_metric = activity_metric.count().to_dict()
edges_count_metric = transition_metric.count().to_dict()
mean_time_node_metric = activity_metric.mean_duration().fillna(0).to_dict()

# Getting the graph
alpha_miner = SimpleMiner(data_holder)
alpha_miner.apply()
graph = alpha_miner.graph

# Adding the metrics to the graph
graph.graph.add_node_metric('count', nodes_count_metric)
graph.graph.add_edge_metric('count', edges_count_metric)
graph.graph.add_node_metric('mean_time', mean_time_node_metric)

AttributeError: ignored

In [ ]:
# Calculation of the metrics
nodes_count_metric = activity_metric.count().to_dict()
edges_count_metric = transition_metric.count().to_dict()
mean_time_node_metric = activity_metric.mean_duration().fillna(0).to_dict()

# Getting the graph
alpha_miner = InductiveMiner(data_holder)
alpha_miner.apply()
graph = alpha_miner.graph

# Adding the metrics to the graph
graph.add_node_metric('count', nodes_count_metric)
graph.add_edge_metric('count', edges_count_metric)
graph.add_node_metric('mean_time', mean_time_node_metric)

TypeError: ignored

In [ ]:
painter = GraphvizPainter()

# Specify which metrics influence the nodes' colors and the edges' widths
painter.apply(graph, node_style_metric='count', edge_style_metric='count')
# or 

# painter.apply(graph, node_style_metric='mean_time', edge_style_metric='count')

painter.write_graph("metric_graph_codes.png", format = 'png')
painter.show()

In [ ]:
from sberpm.autoinsights import AutoInsights

In [ ]:
# Initialization
auto_i = AutoInsights(data_holder, time_unit='day')

In [ ]:
alpha_plus_miner = AlphaPlusMiner(data_holder)

alpha_miner = SimpleMiner(data_holder)

#alpha_miner.apply()

In [ ]:
# Transition duration
auto_i.apply(miner=alpha_miner, mode='time')
graph = auto_i.get_graph()

painter = GraphvizPainter()
painter.apply_insights(graph)
painter.write_graph("metric_graph_codes_insights_alpha.png", format = 'png')
painter.show()

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


#PM4PY

In [ ]:
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery

# viz
from pm4py.visualization.petrinet import visualizer as pn_visualizer
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.visualization.heuristics_net import visualizer as hn_visualizer
from pm4py.visualization.dfg import visualizer as dfg_visualization

In [ ]:
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.causal import algorithm as causal_miner
from pm4py.algo.discovery.dfg import algorithm as dfg
from pm4py.algo.discovery.inductive import algorithm as inductive_miner

In [ ]:
df = pm4py.format_dataframe(burek,case_id='Well number',activity_key='Operation code',timestamp_key='Timestamp')

In [ ]:
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
dfg = dfg_discovery.apply(df)

from pm4py.visualization.dfg import visualizer as dfg_visualization
gviz = dfg_visualization.apply(dfg, log=df, variant=dfg_visualization.Variants.FREQUENCY)
dfg_visualization.view(gviz)

In [ ]:
net, initial_marking, final_marking = inductive_miner.apply(df)

In [ ]:
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
heu_net = heuristics_miner.apply_heu(df, parameters={heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: 0.99})

In [ ]:
from pm4py.algo.discovery.temporal_profile import algorithm as temporal_profile_discovery
parameters = {}
temporal_profile = temporal_profile_discovery.apply(df, parameters=parameters)

In [ ]:
net, initial_marking, final_marking = inductive_miner.apply(df)

from pm4py.algo.conformance.tokenreplay import algorithm as token_replay
from pm4py.algo.evaluation.generalization import algorithm as generalization_evaluator
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator

from pm4py.algo.evaluation.simplicity import algorithm as simplicity_evaluator

replayed_traces_inductive_miner = token_replay.apply(df, net, initial_marking, final_marking)
print('Inductive fitness',replayed_traces_inductive_miner)
gen = generalization_evaluator.apply(df, net, initial_marking, final_marking)
print('Inductive miner generalization',gen)

prec = precision_evaluator.apply(df, net, initial_marking, final_marking, variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN)
print('Inductive precision',prec)

simp = simplicity_evaluator.apply(net)
print('Inductive simplicity',simp)

net, initial_marking, final_marking = alpha_miner.apply(df)

replayed_traces_alpha_miner = token_replay.apply(df, net, initial_marking, final_marking)
print('Alpha fitness',replayed_traces_alpha_miner)

gen = generalization_evaluator.apply(df, net, initial_marking, final_marking)
print('Alpha miner generalization',gen)

prec = precision_evaluator.apply(df, net, initial_marking, final_marking, variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN)
print('Alpha miner precision',prec)

simp = simplicity_evaluator.apply(net)
print('Alpha miner simplicity',simp)

net, im, fm = heuristics_miner.apply(df, parameters={heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: 0.99})


replayed_traces_heu_miner = token_replay.apply(df, net, im, fm)
print('Heuristic fitness',replayed_traces_heu_miner)


gen = generalization_evaluator.apply(df, net, im, fm)
print('Heu miner generalization',gen)

prec = precision_evaluator.apply(df, net, im, fm, variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN)
print('Heuristic miner precision',prec)

simp = simplicity_evaluator.apply(net)
print('Heuristic miner simplicity',simp)

net4,im4,fm4 = pm4py.discover_petri_net_alpha_plus(df)
#pm4py.view_petri_net(net4,im4,fm4)


replayed_traces_alpha_plus = token_replay.apply(df, net4, im4, fm4)
print('Alpha plus fitness',replayed_traces_alpha_plus)

gen = generalization_evaluator.apply(df, net4, im4, fm4)
print('Alpha plus generalization',gen)

prec = precision_evaluator.apply(df, net4, im4, fm4, variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN)
print('Alpha plus miner precision',prec)

simp = simplicity_evaluator.apply(net4)
print('Alpha plus miner simplicity',simp)

replaying log with TBR, completed variants ::   0%|          | 0/2 [00:00<?, ?it/s]

Inductive fitness [{'trace_is_fit': True, 'trace_fitness': 1.0, 'activated_transitions': [(tau_1, None), (init_loop_7, None), (3c2d5194-2854-4f5b-be4b-b0beab22e569, '303'), (37505b7c-19aa-45b3-beef-391002e4f239, '318'), (skip_10, None), (070ccb1c-f8eb-47b1-b1de-7e6c9c2d7a31, '434'), (init_loop_12, None), (init_loop_14, None), (init_loop_16, None), (skip_18, None), (be7ce450-f3e7-43d3-bf4c-cd0de5fde1a8, '201'), (loop_17, None), (skip_18, None), (0d4bf093-8016-4cb0-9420-d00174115ef1, '343'), (loop_17, None), (skip_18, None), (dcbbbb3d-fef0-43aa-b7c5-25e79cf9e372, '425'), (loop_17, None), (skip_18, None), (be7ce450-f3e7-43d3-bf4c-cd0de5fde1a8, '201'), (loop_17, None), (skip_18, None), (0d4bf093-8016-4cb0-9420-d00174115ef1, '343'), (loop_17, None), (skip_18, None), (dcbbbb3d-fef0-43aa-b7c5-25e79cf9e372, '425'), (loop_17, None), (skip_18, None), (42142120-6c45-4b5c-b597-20de5f09d2ec, '402'), (loop_17, None), (skip_18, None), (be7ce450-f3e7-43d3-bf4c-cd0de5fde1a8, '201'), (loop_17, None), (s

replaying log with TBR, completed variants ::   0%|          | 0/2 [00:00<?, ?it/s]

Inductive miner generalization 0.4441083489769839


replaying log with TBR, completed variants ::   0%|          | 0/174 [00:00<?, ?it/s]

...

Alpha plus fitness [{'trace_is_fit': False, 'trace_fitness': 0.7614898506319419, 'activated_transitions': [(artificial_start, None), (303, '303'), (318, '318'), (434, '434'), (201, '201'), (343, '343'), (425, '425'), (201, '201'), (343, '343'), (425, '425'), (402, '402'), (201, '201'), (425, '425'), (401, '401'), (316, '316'), (308, '308'), (303, '303'), (318, '318'), (303, '303'), (318, '318'), (419, '419'), (401, '401'), (415, '415'), (201, '201'), (419, '419'), (401, '401'), (343, '343'), (415, '415'), (201, '201'), (419, '419'), (401, '401'), (343, '343'), (415, '415'), (201, '201'), (420, '420'), (416, '416'), (1002, '1002'), (201, '201'), (419, '419'), (416, '416'), (420, '420'), (343, '343'), (201, '201'), (343, '343'), (425, '425'), (416, '416'), (401, '401'), (201, '201'), (343, '343'), (425, '425'), (416, '416'), (401, '401'), (201, '201'), (343, '343'), (425, '425'), (416, '416'), (401, '401'), (201, '201'), (343, '343'), (425, '425'), (416, '416'), (401, '401'), (201, '201'

replaying log with TBR, completed variants ::   0%|          | 0/2 [00:00<?, ?it/s]

Alpha plus generalization 0.4507997516905098


replaying log with TBR, completed variants ::   0%|          | 0/174 [00:00<?, ?it/s]

Alpha plus miner precision 0.0
Alpha plus miner simplicity 0.18181818181818182
